In [1]:
# Used for plotting data
%matplotlib inline
import matplotlib.pyplot as plt
# Used for data storage and manipulation 
import numpy as np
import pandas as pd
# Used for Regression Modelling
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.model_selection import train_test_split
# Used for Acc metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
# For stepwise regression
import statsmodels.api as sm
# box plots
import seaborn as sns
# pairplot
from seaborn import pairplot
# Correlation plot
from statsmodels.graphics.correlation import plot_corr
%autosave 60

Autosaving every 60 seconds


In [19]:
# Load your data 
data = pd.read_csv("spreadspoke_scores.csv")
# Check out shape
print(data.shape)

no_2020 = data[(data['schedule_season'] < 2020)]
current = no_2020[(no_2020['schedule_season'] > 2005)]

# Adding .head() to your dataset allows you to see the first rows in the dataset. Input a number inside the brackets to specificy how many rows you want returned or else 5 rows are returned.
current.head(10)

#If you want to see the last rows in the data set then change data.head() to data.tail()

(12934, 17)


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
8940,9/7/2006,2006,1,False,Pittsburgh Steelers,28.0,17.0,Miami Dolphins,PIT,-1.5,34.5,Heinz Field,False,62.0,3.0,75,NaN
8941,9/10/2006,2006,1,False,Arizona Cardinals,34.0,27.0,San Francisco 49ers,ARI,-9.5,44,University of Phoenix Stadium,False,72.0,0.0,NaN,DOME
8942,9/10/2006,2006,1,False,Carolina Panthers,6.0,20.0,Atlanta Falcons,CAR,-4.5,39,Bank of America Stadium,False,73.0,6.0,76,NaN
8943,9/10/2006,2006,1,False,Cleveland Browns,14.0,19.0,New Orleans Saints,CLE,-3.0,36,FirstEnergy Stadium,False,65.0,13.0,74,NaN
8944,9/10/2006,2006,1,False,Detroit Lions,6.0,9.0,Seattle Seahawks,SEA,-6.0,44,Ford Field,False,72.0,0.0,NaN,DOME
8945,9/10/2006,2006,1,False,Green Bay Packers,0.0,26.0,Chicago Bears,CHI,-3.5,35,Lambeau Field,False,56.0,11.0,62,NaN
8946,9/10/2006,2006,1,False,Houston Texans,10.0,24.0,Philadelphia Eagles,PHI,-6.0,37,Reliant Stadium,False,72.0,0.0,NaN,DOME
8947,9/10/2006,2006,1,False,Jacksonville Jaguars,24.0,17.0,Dallas Cowboys,DAL,-2.5,36.5,EverBank Field,False,80.0,10.0,75,NaN
8948,9/10/2006,2006,1,False,Kansas City Chiefs,10.0,23.0,Cincinnati Bengals,CIN,-1.5,49.5,Arrowhead Stadium,False,69.0,11.0,83,NaN
8949,9/10/2006,2006,1,False,New England Patriots,19.0,17.0,Buffalo Bills,NE,-10.0,41,Gillette Stadium,False,64.0,13.0,62,NaN


In [20]:
clean_current = current[['schedule_season', 'team_home', 'score_home', 'score_away', 'team_away', 'team_favorite_id', 'spread_favorite', 'over_under_line']]
clean_current.head()

,schedule_season,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line
8940,2006,Pittsburgh Steelers,28.0,17.0,Miami Dolphins,PIT,-1.5,34.5
8941,2006,Arizona Cardinals,34.0,27.0,San Francisco 49ers,ARI,-9.5,44
8942,2006,Carolina Panthers,6.0,20.0,Atlanta Falcons,CAR,-4.5,39
8943,2006,Cleveland Browns,14.0,19.0,New Orleans Saints,CLE,-3.0,36
8944,2006,Detroit Lions,6.0,9.0,Seattle Seahawks,SEA,-6.0,44


In [22]:
#print(sorted(clean_current["team_home"].unique()))
#print(sorted(clean_current["team_favorite_id"].unique()))

# doing the inane and cleaning the data so the team names match the abbreviations used in team_favorite_id
# this is for ease of comparison later on
for index, row in clean_current.iterrows():
    if row['team_home'] == "Arizona Cardinals":
        clean_current.at[index, 'team_home'] = "ARI"
    elif row['team_home'] == "Atlanta Falcons":
        clean_current.at[index, 'team_home'] = "ATL"
    elif row['team_home'] == "Baltimore Ravens":
        clean_current.at[index, 'team_home'] = "BAL"
    elif row['team_home'] == "Buffalo Bills":
        clean_current.at[index, 'team_home'] = "BUF"
    elif row['team_home'] == "Carolina Panthers":
        clean_current.at[index, 'team_home'] = "CAR"
    elif row['team_home'] == "Chicago Bears":
        clean_current.at[index, 'team_home'] = "CHI"
    elif row['team_home'] == "Cincinnati Bengals":
        clean_current.at[index, 'team_home'] = "CIN"
    elif row['team_home'] == "Cleveland Browns":
        clean_current.at[index, 'team_home'] = "CLE"
    elif row['team_home'] == "Dallas Cowboys":
        clean_current.at[index, 'team_home'] = "DAL"
    elif row['team_home'] == "Denver Broncos":
        clean_current.at[index, 'team_home'] = "DEN"
    elif row['team_home'] == "Detroit Lions":
        clean_current.at[index, 'team_home'] = "DET"
    elif row['team_home'] == "Green Bay Packers":
        clean_current.at[index, 'team_home'] = "GB"
    elif row['team_home'] == "Houston Texans":
        clean_current.at[index, 'team_home'] = "HOU"
    elif row['team_home'] == "Indianapolis Colts":
        clean_current.at[index, 'team_home'] = "IND"
    elif row['team_home'] == "Jacksonville Jaguars":
        clean_current.at[index, 'team_home'] = "JAX"
    elif row['team_home'] == "Kansas City Chiefs":
        clean_current.at[index, 'team_home'] = "KC"
    elif row['team_home'] == "Los Angeles Chargers" or row['team_home'] == "San Diego Chargers":
        clean_current.at[index, 'team_home'] = "LAC" # regardless of when team was in LA or SD, team ID is LAC
    elif row['team_home'] == "Los Angeles Rams" or row['team_home'] == "St. Louis Rams":
        clean_current.at[index, 'team_home'] = "LAR" # regardless of when team was in LA or STL, team ID is LAR
    elif row['team_home'] == "Miami Dolphins":
        clean_current.at[index, 'team_home'] = "MIA"
    elif row['team_home'] == "Minnesota Vikings":
        clean_current.at[index, 'team_home'] = "MIN"
    elif row['team_home'] == "New England Patriots":
        clean_current.at[index, 'team_home'] = "NE"
    elif row['team_home'] == "New Orleans Saints":
        clean_current.at[index, 'team_home'] = "NO"
    elif row['team_home'] == "New York Giants":
        clean_current.at[index, 'team_home'] = "NYG"
    elif row['team_home'] == "New York Jets":
        clean_current.at[index, 'team_home'] = "NYJ"
    elif row['team_home'] == "Oakland Raiders":
        clean_current.at[index, 'team_home'] = "LVR" # our data only has the Oakland Raiders, but team ID is still LVR b/c they're in Vegas now
    elif row['team_home'] == "Philadelphia Eagles":
        clean_current.at[index, 'team_home'] = "PHI"
    elif row['team_home'] == "Pittsburgh Steelers":
        clean_current.at[index, 'team_home'] = "PIT"
    elif row['team_home'] == "San Francisco 49ers":
        clean_current.at[index, 'team_home'] = "SF"
    elif row['team_home'] == "Seattle Seahawks":
        clean_current.at[index, 'team_home'] = "SEA"
    elif row['team_home'] == "Tampa Bay Buccaneers":
        clean_current.at[index, 'team_home'] = "TB"
    elif row['team_home'] == "Tennessee Titans":
        clean_current.at[index, 'team_home'] = "TEN"
    elif row['team_home'] == "Washington Redskins":
        clean_current.at[index, 'team_home'] = "WAS"
    # ------------------------------------------------------ now for away
    if row['team_away'] == "Arizona Cardinals":
        clean_current.at[index, 'team_away'] = "ARI"
    elif row['team_away'] == "Atlanta Falcons":
        clean_current.at[index, 'team_away'] = "ATL"
    elif row['team_away'] == "Baltimore Ravens":
        clean_current.at[index, 'team_away'] = "BAL"
    elif row['team_away'] == "Buffalo Bills":
        clean_current.at[index, 'team_away'] = "BUF"
    elif row['team_away'] == "Carolina Panthers":
        clean_current.at[index, 'team_away'] = "CAR"
    elif row['team_away'] == "Chicago Bears":
        clean_current.at[index, 'team_away'] = "CHI"
    elif row['team_away'] == "Cincinnati Bengals":
        clean_current.at[index, 'team_away'] = "CIN"
    elif row['team_away'] == "Cleveland Browns":
        clean_current.at[index, 'team_away'] = "CLE"
    elif row['team_away'] == "Dallas Cowboys":
        clean_current.at[index, 'team_away'] = "DAL"
    elif row['team_away'] == "Denver Broncos":
        clean_current.at[index, 'team_away'] = "DEN"
    elif row['team_away'] == "Detroit Lions":
        clean_current.at[index, 'team_away'] = "DET"
    elif row['team_away'] == "Green Bay Packers":
        clean_current.at[index, 'team_away'] = "GB"
    elif row['team_away'] == "Houston Texans":
        clean_current.at[index, 'team_away'] = "HOU"
    elif row['team_away'] == "Indianapolis Colts":
        clean_current.at[index, 'team_away'] = "IND"
    elif row['team_away'] == "Jacksonville Jaguars":
        clean_current.at[index, 'team_away'] = "JAX"
    elif row['team_away'] == "Kansas City Chiefs":
        clean_current.at[index, 'team_away'] = "KC"
    elif row['team_away'] == "Los Angeles Chargers" or row['team_away'] == "San Diego Chargers":
        clean_current.at[index, 'team_away'] = "LAC" # regardless of when team was in LA or SD, team ID is LAC
    elif row['team_away'] == "Los Angeles Rams" or row['team_away'] == "St. Louis Rams":
        clean_current.at[index, 'team_away'] = "LAR" # regardless of when team was in LA or STL, team ID is LAR
    elif row['team_away'] == "Miami Dolphins":
        clean_current.at[index, 'team_away'] = "MIA"
    elif row['team_away'] == "Minnesota Vikings":
        clean_current.at[index, 'team_away'] = "MIN"
    elif row['team_away'] == "New England Patriots":
        clean_current.at[index, 'team_away'] = "NE"
    elif row['team_away'] == "New Orleans Saints":
        clean_current.at[index, 'team_away'] = "NO"
    elif row['team_away'] == "New York Giants":
        clean_current.at[index, 'team_away'] = "NYG"
    elif row['team_away'] == "New York Jets":
        clean_current.at[index, 'team_away'] = "NYJ"
    elif row['team_away'] == "Oakland Raiders":
        clean_current.at[index, 'team_away'] = "LVR" # our data only has the Oakland Raiders, but team ID is still LVR b/c they're in Vegas now
    elif row['team_away'] == "Philadelphia Eagles":
        clean_current.at[index, 'team_away'] = "PHI"
    elif row['team_away'] == "Pittsburgh Steelers":
        clean_current.at[index, 'team_away'] = "PIT"
    elif row['team_away'] == "San Francisco 49ers":
        clean_current.at[index, 'team_away'] = "SF"
    elif row['team_away'] == "Seattle Seahawks":
        clean_current.at[index, 'team_away'] = "SEA"
    elif row['team_away'] == "Tampa Bay Buccaneers":
        clean_current.at[index, 'team_away'] = "TB"
    elif row['team_away'] == "Tennessee Titans":
        clean_current.at[index, 'team_away'] = "TEN"
    elif row['team_away'] == "Washington Redskins":
        clean_current.at[index, 'team_away'] = "WAS"
# damn that was a lot, wonder if that can be simplified somehow lmao

# --------------------------------------------------------